<a href="https://colab.research.google.com/github/Tunguyen09CNTT03/BCCK/blob/main/Biometrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ví dụ về nhận diện sinh trắc học

Nhận diện dấu vân tay


# **Trích xuất đặc điểm vân tay**
Giới thiệu:
Sinh trắc vân tay bao gồm: Thu thập hình ảnh, Nâng cao hình ảnh, Trích xuất tính năng và khớp với mẫu. Vì tập dữ liệu có dấu vân tay duy nhất nên tôi sẽ triển khai trích xuất tính năng và các kỹ thuật khác nhau để nâng cao hình ảnh như Phát hiện cạnh, ngưỡng thích ứng. Trích xuất tính năng cấu thành phát hiện Ridge (tính năng cấp 1) và trích xuất Minutiae (tính năng cấp 3) để tạo mẫu sau đó hình ảnh truy vấn được đối sánh bằng cách sử dụng đường cong ROC AUC số liệu.

# Nói về bộ dữ liệu dấu vân tay
Bộ dữ liệu vân tay Sokoto Coventry (SOCOFing) là cơ sở dữ liệu vân tay sinh trắc học được thiết kế cho mục đích nghiên cứu học thuật. SOCOFing được tạo thành từ 6.000 hình ảnh dấu vân tay từ 600 đối tượng châu Phi và chứa các thuộc tính độc đáo như nhãn cho giới tính, tên bàn tay và ngón tay cũng như các phiên bản được thay đổi tổng hợp với ba mức độ thay đổi khác nhau để xóa, xoay trung tâm và cắt chữ z.

Phụ thuộc và dữ liệu

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'socofing:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F38300%2F58521%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240422%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240422T143456Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6625eba10cfca376994b6744d1e729a978f39175ef4cca5c1ed1528ee4f1a9a8269cc900372d915aa6460fc4eb2e97d9e73046cdc8f426276f82f5cc01f63af5173a077fd60147f82a52da74f01a8dbf07bce1066c4983d414fd94f1999df000bdcdf11cdedcee694911ec649dd069c9fcafe842a090b31fe1662cc239b80a46b4ef372c83fab6b97cb2d9198d288a17a1472e48e2be5f7c54340774cd0f6c0b15c21ac23ca1340870a769418394e9b0ab352075d1b73cc5ef4aebf73d96076226a8c633e298721ee833cdffb82b9340b671e4d5d7d997aedd57cbd578c8920f18c28d30248c936f983979af1d2b23e5a764325bf5ccabfd68e952a9da48ecac'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

In [ ]:
import numpy as np
import glob
import random
import imageio
import PIL, cv2
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from skimage.morphology import convex_hull_image, erosion
from skimage.morphology import square
import matplotlib.image as mpimg
import skimage
import math
from scipy.ndimage.filters import convolve
from PIL import Image,ImageFilter
from skimage.feature import hessian_matrix, hessian_matrix_eigvals

In [ ]:
# DỮ LIỆU VÂN TAY KAGGLE

DATA_DIR = "../input/socofing/SOCOFing/Real/"
list_dirs = list(glob.glob(DATA_DIR+"*.BMP"))
num_images = len(list_dirs)

Hiển thị hình ảnh ngẫu nhiên từ dữ liệu

In [ ]:
random.seed(42)

r = random.randint(0,num_images)
display_list = list_dirs[r:r+3]

image1 = imageio.imread(display_list[0])
image2 = imageio.imread(display_list[1])
image3 = imageio.imread(display_list[2])

fig, axes = plt.subplots(1,3,figsize = (16,16));
axes[0].imshow(image1);
axes[1].imshow(image2);
axes[2].imshow(image3);

Biến đổi hình ảnh


1.   Làm mịn hình ảnh
2.   Ngưỡng
3.   Phát hiện cạnh




Các kỹ thuật tiền xử lý và nâng cao hình ảnh như làm mịn, tạo ngưỡng và phát hiện cạnh được sử dụng để làm cho các đặc điểm nổi bật hơn trong dữ liệu nhằm trích xuất chính xác hơn.

In [ ]:
gauss_blur = cv2.GaussianBlur(image1,(1,1),0)
median_blur = cv2.medianBlur(image1,1)

fig, axes = plt.subplots(1,3,figsize = (16,16));
axes[0].set_title("original Image");
axes[0].imshow(image1);
axes[1].set_title("Gaussian Blurred Image");
axes[1].imshow(gauss_blur);
axes[2].set_title("Median Blurred Image");
axes[2].imshow(median_blur);

Biểu đồ

In [ ]:
fig, axes = plt.subplots(1,3,figsize = (18,5))
axes[0].hist(image1.ravel(), bins=256, color ="r");
axes[1].hist(image2.ravel(), bins=256);
axes[2].hist(image3.ravel(), bins=256, color ="g");

Dữ liệu ở dạng nhị phân - thực hiện ngưỡng trung bình và ngưỡng thích ứng

In [ ]:
# ngưỡng trung bình - cho kết quả xấu
THRESHOLD1 = image1.mean()
THRESHOLD2 = image2.mean()
THRESHOLD3 = image3.mean()

image1 = np.array(image1 > THRESHOLD1).astype(int) * 255
image2 = np.array(image2 > THRESHOLD2).astype(int) * 254
image3 = np.array(image3 > THRESHOLD3).astype(int) * 254

fig, axes = plt.subplots(1,3,figsize = (16,16));
axes[0].imshow(image1);
axes[1].imshow(image2);
axes[2].imshow(image3);

In [ ]:
# Ngưỡng thích ứng từ thư viện OpenCV - tốt hơn Ngưỡng trung bình

img1 = cv2.imread(display_list[0],0)
img2 = cv2.imread(display_list[1],0)
img3 = cv2.imread(display_list[2],0)

# Ngưỡng cửa của Otsu
ret1,th1 = cv2.threshold(img1,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
ret2,th2 = cv2.threshold(img2,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
ret3,th3 = cv2.threshold(img3,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

fig, axes = plt.subplots(1,3,figsize = (12,12));
axes[0].set_title("Otsu's thresholding - Image 1");
axes[0].imshow(th2);
axes[1].set_title("Otsu's thresholding - Image 2");
axes[1].imshow(th2);
axes[2].set_title("Otsu's thresholding - Image 3");
axes[2].imshow(th2);

Phát hiện cạnh:

In [ ]:
# Chuyển đổi sang thang độ xám
img_name = display_list[0]
gray_img_array = np.array(Image.open(img_name).convert('P'))

In [ ]:
vertical_robert_filter = np.array([[1,0],[0,-1]])
horizontal_robert_filter = np.array([[0,1],[-1,0]])

vertical_sobel_filter = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
horizontal_sobel_filter = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])

vertical_prewitt_filter = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
horizontal_prewitt_filter = np.array([[-1,-1,-1],[0,0,0],[1,1,1]])

print("vertical robert filter\n",vertical_robert_filter )
print("horizontal robert filter\n",horizontal_robert_filter)
print("vertical sobel filter: \n", vertical_sobel_filter)
print("horizontal sobel filter: \n", horizontal_sobel_filter)

print("vertical prewitt filter: \n", vertical_prewitt_filter)
print("horizontal prewitt filter: \n", horizontal_prewitt_filter)

In [ ]:
# Thực hiện:
gray_img = Image.fromarray(gray_img_array)

convolved_img1 = convolve(gray_img,vertical_robert_filter)
convolved_img1 = convolve(convolved_img1,horizontal_robert_filter)

convolved_img2 = convolve(gray_img,vertical_sobel_filter)
convolved_img2 = convolve(convolved_img2,horizontal_sobel_filter)

convolved_img3 =  convolve(gray_img,vertical_prewitt_filter )
convolved_img3 =  convolve(gray_img,horizontal_prewitt_filter )

In [ ]:
fig, axes = plt.subplots(1,3,figsize = (12,12));
axes[0].set_title("Robert");
axes[0].imshow(convolved_img1);
axes[1].set_title("Sobel");
axes[1].imshow(convolved_img2);
axes[2].set_title("Prewitt");
axes[2].imshow(convolved_img3);

Phát hiện Ridge

In [ ]:
src_path = img_name

def detect_ridges(gray, sigma= 0.1):
    H_elems = hessian_matrix(gray, sigma=sigma, order='rc')
    maxima_ridges, minima_ridges = hessian_matrix_eigvals(H_elems)
    return maxima_ridges, minima_ridges

def plot_images(*images):
    images = list(images)
    n = len(images)
    fig, ax = plt.subplots(ncols=n, sharey=True, figsize = (12,12))
    for i, img in enumerate(images):
        ax[i].imshow(img, cmap='gray')
        ax[i].axis('off')
    plt.subplots_adjust(left=0.03, bottom=0.03, right=0.97, top=0.97)
    plt.show()

img = cv2.imread(src_path, 0) # 0 imports a grayscale
if img is None:
    raise(ValueError(f"Image didn\'t load. Check that '{src_path}' exists."))

a, b = detect_ridges(img, sigma=0.15)

plot_images(img, a, b)

Phát hiện kết thúc và phân nhánh và trích xuất chi tiết
Mã đã cho trích xuất các tính năng như Chấm dứt, Phân nhánh và Chi tiết từ dấu vân tay, đầu ra được hiển thị bên dưới mã:

In [ ]:
def getTerminationBifurcation(img, mask):
    img = img == 255;
    (rows, cols) = img.shape;
    minutiaeTerm = np.zeros(img.shape);
    minutiaeBif = np.zeros(img.shape);

    for i in range(1,rows-1):
        for j in range(1,cols-1):
            if(img[i][j] == 1):
                block = img[i-1:i+2,j-1:j+2];
                block_val = np.sum(block);
                if(block_val == 2):
                    minutiaeTerm[i,j] = 1;
                elif(block_val == 4):
                    minutiaeBif[i,j] = 1;

    mask = convex_hull_image(mask>0)
    mask = erosion(mask, square(5))
    minutiaeTerm = np.uint8(mask)*minutiaeTerm
    return(minutiaeTerm, minutiaeBif)

In [ ]:
class MinutiaeFeature(object):
    def __init__(self, locX, locY, Orientation, Type):
        self.locX = locX;
        self.locY = locY;
        self.Orientation = Orientation;
        self.Type = Type;

def computeAngle(block, minutiaeType):
    angle = 0
    (blkRows, blkCols) = np.shape(block);
    CenterX, CenterY = (blkRows-1)/2, (blkCols-1)/2
    if(minutiaeType.lower() == 'termination'):
        sumVal = 0;
        for i in range(blkRows):
            for j in range(blkCols):
                if((i == 0 or i == blkRows-1 or j == 0 or j == blkCols-1) and block[i][j] != 0):
                    angle = -math.degrees(math.atan2(i-CenterY, j-CenterX))
                    sumVal += 1
                    if(sumVal > 1):
                        angle = float('nan');
        return(angle)
    elif(minutiaeType.lower() == 'bifurcation'):
        (blkRows, blkCols) = np.shape(block);
        CenterX, CenterY = (blkRows - 1) / 2, (blkCols - 1) / 2
        angle = []
        sumVal = 0;
        for i in range(blkRows):
            for j in range(blkCols):
                if ((i == 0 or i == blkRows - 1 or j == 0 or j == blkCols - 1) and block[i][j] != 0):
                    angle.append(-math.degrees(math.atan2(i - CenterY, j - CenterX)))
                    sumVal += 1
        if(sumVal != 3):
            angle = float('nan')
        return(angle)


def extractMinutiaeFeatures(skel, minutiaeTerm, minutiaeBif):
    FeaturesTerm = []

    minutiaeTerm = skimage.measure.label(minutiaeTerm, connectivity=2);
    RP = skimage.measure.regionprops(minutiaeTerm)

    WindowSize = 2
    FeaturesTerm = []
    for i in RP:
        (row, col) = np.int16(np.round(i['Centroid']))
        block = skel[row-WindowSize:row+WindowSize+1, col-WindowSize:col+WindowSize+1]
        angle = computeAngle(block, 'Termination')
        FeaturesTerm.append(MinutiaeFeature(row, col, angle, 'Termination'))

    FeaturesBif = []
    minutiaeBif = skimage.measure.label(minutiaeBif, connectivity=2);
    RP = skimage.measure.regionprops(minutiaeBif)
    WindowSize = 1
    for i in RP:
        (row, col) = np.int16(np.round(i['Centroid']))
        block = skel[row-WindowSize:row+WindowSize+1, col-WindowSize:col+WindowSize+1]
        angle = computeAngle(block, 'Bifurcation')
        FeaturesBif.append(MinutiaeFeature(row, col, angle, 'Bifurcation'))
    return(FeaturesTerm, FeaturesBif)

def ShowResults(skel, TermLabel, BifLabel):
    minutiaeBif = TermLabel * 0;
    minutiaeTerm = BifLabel * 0;

    (rows, cols) = skel.shape
    DispImg = np.zeros((rows, cols, 3), np.uint8)
    DispImg[:, :, 0] = skel;
    DispImg[:, :, 1] = skel;
    DispImg[:, :, 2] = skel;

    RP = skimage.measure.regionprops(BifLabel)
    for idx, i in enumerate(RP):
        (row, col) = np.int16(np.round(i['Centroid']))
        minutiaeBif[row, col] = 1;
        (rr, cc) = skimage.draw.circle_perimeter(row, col, 1);
        skimage.draw.set_color(DispImg, (rr, cc), (255, 0, 0));

    RP = skimage.measure.regionprops(TermLabel)
    for idx, i in enumerate(RP):
        (row, col) = np.int16(np.round(i['Centroid']))
        minutiaeTerm[row, col] = 1;
        (rr, cc) = skimage.draw.circle_perimeter(row, col, 1);
        skimage.draw.set_color(DispImg, (rr, cc), (0, 0, 255));

    plt.figure(figsize=(6,6))
    plt.title("Minutiae extraction results")
    plt.imshow(DispImg)

In [ ]:

img_name = display_list[1]
img = cv2.imread(img_name,0);
img = np.array(img > THRESHOLD1).astype(int)
skel = skimage.morphology.skeletonize(img)
skel = np.uint8(skel)*255;
mask = img*255;

(minutiaeTerm, minutiaeBif) = getTerminationBifurcation(skel, mask);
FeaturesTerm, FeaturesBif = extractMinutiaeFeatures(skel, minutiaeTerm, minutiaeBif)
BifLabel = skimage.measure.label(minutiaeBif, connectivity=1);
TermLabel = skimage.measure.label(minutiaeTerm, connectivity=1);
ShowResults(skel, TermLabel, BifLabel)


Nhận diện khuôn mặt


# Nói về bộ dữ liệu
 Labeled Faces in the Wild, một cơ sở dữ liệu các bức ảnh khuôn mặt được thiết kế để nghiên cứu vấn đề nhận dạng khuôn mặt không bị hạn chế. Bộ dữ liệu chứa hơn 13.000 hình ảnh khuôn mặt được thu thập từ web. Mỗi khuôn mặt đã được dán nhãn với tên của người trong hình. 1680 người trong ảnh có hai hoặc nhiều ảnh khác biệt trong tập dữ liệu. Hạn chế duy nhất trên những khuôn mặt này là chúng được phát hiện bởi máy dò khuôn mặt Viola-Jones.



140k Khuôn mặt thật và giả: Bộ dữ liệu này bao gồm tất cả 70k khuôn mặt THẬT từ bộ dữ liệu Flickr do Nvidia thu thập, cũng như 70k khuôn mặt giả được lấy mẫu từ 1 triệu khuôn mặt FAKE (do StyleGAN tạo ra) do Bojan cung cấp.

Trong tập dữ liệu này đã kết hợp cả hai tập dữ liệu, thay đổi kích thước tất cả các hình ảnh thành 256px và chia dữ liệu thành train, validation và test set. Có bao gồm một số tệp CSV để thuận tiện.

facenet pytorch vggface2: Ngữ cảnh
đào tạo trước để phát hiện và nhận dạng khuôn mặt.

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'lfwpeople:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F19136%2F796646%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240422%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240422T144056Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0f1d55b4c439f10c16f4135bfb37f84190726d593176b9593a2eae28e9a7ab40405ea2d3a65b10d8bd4496c72700ccef3938ef8d50275a57e1d1d5cd16fb3a3eedfba215ac066edff27c8af2a6e75762108b9c3a55c97790793c6ce2a19584d170667592a777efa06b508c371f4c4ded4e2d285f32c4e722983e0223889950c6c792fb8b4c8c1da8a880a760c3986e6f0682a08fc74d2033632a1deb1b896f0b713af9d4bd798dcfae747ed6866d844f5fa4ce456b961b3e540683284f428196a47eb0de180fe400f2bc9fa1ca7cb4afdaeb999b3c6b7a3a9fd3101cf3ec8607afe99284d901d8e502dbc573091b763dcccfb7e91a0b530857b14dccb8b5b750,140k-real-and-fake-faces:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F501529%2F939937%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240422%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240422T144056Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8f4b9539d67b16057c2b354454a696b2478a5250df1e4ec3f657eb8f19a978fac93e4ee682cd2c3635af258c8a878ede6db8645ea09c037f3d6b7f7b5796515fe3ef43ef7805b82f9fbe20e9c22fa2c6bf15dfba5da70ba519dfec2098f18ad66a160a5699862f8f3885b4195987aa778a838d0223bdbbed9385f56a139a093180ac0ee159823e0ebc8b6fa0655bd00dd21bf53e26717ab62826b8268a110c027331f5d7c39b00f739bf32eaf197bbc23dabd46c9760cf7978cdde607eb8e624de46cca7b5f881af8d5f99c518a3135d76e5647e0b0fd9b8e2cca52f42dc94d663758f9cf0e9d6e13213d4c79c8a8095073c107873d843b7d926b0e909c498f5,facenet-pytorch-vggface2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F442595%2F1003630%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240422%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240422T144056Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D34be9aefb3758c63a7ab8bfb1b157438f951564df00a38f2cd5a7dcad912f5b53388d01c1da8dc083dbb8e23e153290235b5c33884e3f9c0b2f38a85367f2557664e98b2c54be875765944bf31274fd64319c0fe1624b1d9fc14152a867c20cc4472d65cb280de7f6beacd5287f5838596c07b30a399e36a7ab7912a8e0257f5e6139e8d3762de0a646cbe51b0366432df27595037bdaa9e29b910e2af0144ceb43a56c796643656ad513e89d351bb9f114e65a4464b0678a4ff93c46ef42eeaaf3be66d0cdc9e2f6b52e2c46b1b3003ba207cdd54b92ba890baa2cbfc19e4cf01f0600361196c14b44df6e05f502569b1af7026047e479c600b2015c059b603,face-recognition-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1365967%2F2415961%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240422%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240422T144056Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da5c6535d04b7f8a88f0ac97f740695fe8ceb4f981071a8976f373a2bb72be36f9e7b66404c4019587c657fbe2ee569946793f50b2d9e5754a255ae6566feee41d6b3bc5e8a276116ec8f3674cc662de0247eb8db07494b02ff8969814af7d4c777698b19fff321dd3016cddb20d450f6ed539aea187f3b5c44bc04e08626a3a0e9e8fbb048d17330bdc24081059b5485b35d3417e2d3e76167755af569b2021e6049d8a0fce9cbca5eedc24fdbbc058dec8e8f1bfaf821ff4377346958afbd6c5d3bead0c8f3cf1d765c5cb322ad3805a10037c123850e55523e54bb094a9b3c488c97586274f95306a72c8397a1e4d2dd7890f0eb52a23b75c9025a51093fb0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

Nhận dạng khuôn mặt
sử dụng tính năng nhúng khuôn mặt

In [ ]:
%%capture
!pip install mtcnn
!pip install facenet-pytorch

In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
import random
from scipy.spatial.distance import cosine
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import Dataset, Subset
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from facenet_pytorch import MTCNN, InceptionResnetV1
%matplotlib inline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
test_dir = '/kaggle/working/test-faces'
base_path = '/kaggle/input/face-recognition-dataset/Face Data/Face Dataset'
target_path = '/kaggle/working/need-faces'
root_path = '/kaggle/working/need-faces'
no_real = '/kaggle/working/no-real'

In [ ]:
def create_root_dir(base_path, target_path):
    if not os.path.exists(target_path):
        os.makedirs(target_path)
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        if os.path.isdir(folder_path):
            num_images = len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])
            if num_images > 3:
                target_folder_path = os.path.join(target_path, folder_name)
                shutil.copytree(folder_path, target_folder_path)
    print(f"Các thư mục con có hơn 3 hình ảnh đã được sao chép vào {target_path}")


def create_test_dir(base_path, target_path):
    if not os.path.exists(target_path):
        os.makedirs(target_path)
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        if os.path.isdir(folder_path):
            num_images = len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])
            if num_images == 3:
                target_folder_path = os.path.join(target_path, folder_name)
                shutil.copytree(folder_path, target_folder_path)
    print(f"Các thư mục con với 3 hình ảnh đã được sao chép vào {target_path}")

In [ ]:
create_root_dir(base_path, target_path)
create_test_dir(base_path, no_real)

In [ ]:
class FacesDataset(Dataset):
    def __init__(self, root_dir, transform=None, mtcnn=None):
        # Đi qua tất cả các thư mục con và thu thập thông tin về hình ảnh
        self.transform = transform
        self.mtcnn = mtcnn
        self.file_list = []
        self.labels = []
        all_labels = []  # Danh sách để lưu trữ tất cả các nhãn cho LabelEncoder

        # Đi qua tất cả các thư mục con và thu thập thông tin về hình ảnh
        for folder_name in os.listdir(root_dir):
            folder_path = os.path.join(root_dir, folder_name)
            label = int(folder_name)  # Chuyển đổi tên thư mục thành nhãn
            all_labels.append(label)  # Thêm thẻ vào danh sách tất cả các thẻ
            for filename in os.listdir(folder_path):
                if filename.lower().endswith(('.jpg')):
                    self.file_list.append(os.path.join(folder_path, filename))
                    self.labels.append(label)

        # Khởi tạo và đào tạo LabelEncoder
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(all_labels)
        # Chuyển đổi tất cả nhãn thành chỉ mục
        self.encoded_labels = self.label_encoder.transform(self.labels)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path).convert("RGB")
        if self.mtcnn:
            boxes, _ = self.mtcnn.detect(img)
            if boxes is not None:
                box = boxes[0]
                img = img.crop((box[0], box[1], box[2], box[3]))
            else:
                raise Exception(f"Face not detected by MTCNN in file: {img_path}")

        label = self.encoded_labels[index]

        if self.transform:
            img = self.transform(img)

        return img, label

def visualize_samples(dataset, indices, title=None, count=10):
    plt.figure(figsize=(count*3, 3))
    display_indices = indices[:count]
    if title:
        plt.suptitle(title)
    for i, index in enumerate(display_indices):
        x, y = dataset[index]
        plt.subplot(1, count, i+1)
        plt.title(f"ID: {y}")
        plt.imshow(x)
        plt.grid(False)
        plt.axis('off')
    plt.show()
# Sử dụng elemets này
dataset = FacesDataset(root_path)
visualize_samples(dataset, range(len(dataset)), title="Faces")

In [ ]:
mtcnn = MTCNN(device=device, image_size=160, keep_all=False)

batch_size = 64
train_dataset = FacesDataset(root_path,transform = transforms.Compose([
                                                    transforms.Resize((160, 160)),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                                ]), mtcnn=mtcnn)
control_dataset = FacesDataset(no_real,transform = transforms.Compose([
                                                    transforms.Resize((160, 160)),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                                ]), mtcnn=mtcnn)

indices_per_label = defaultdict(list)
for idx in range(len(dataset)):
    _, label = dataset[idx]
    indices_per_label[label].append(idx)

selected_indices = []
remaining_indices = []
for label, indices in indices_per_label.items():
    selected_index = random.choice(indices)
    selected_indices.append(selected_index)

    remaining_indices.extend([idx for idx in indices if idx != selected_index])

# Tạo tập dữ liệu tập hợp con với các chỉ mục đã chọn và các chỉ mục còn lại
subset_dataset_selected = Subset(train_dataset, selected_indices)
subset_dataset_remaining = Subset(train_dataset, remaining_indices)

train_loader = torch.utils.data.DataLoader(subset_dataset_remaining, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(subset_dataset_selected, batch_size=batch_size)
control_loader = torch.utils.data.DataLoader(control_dataset, batch_size=batch_size)

In [ ]:
def get_embeddings(model, data_loader):
    model.eval()  # Chuyển mô hình sang chế độ đánh giá
    embeddings = []  # Nhúng danh sách lưu trữ
    labels = []  # Danh sách lưu trữ nhãn

    with torch.no_grad():  # Vô hiệu hóa tính toán gradient
        for images, batch_labels in data_loader:
            images = images.to(device)  # Chuyển hình ảnh sang thiết bị
            batch_embeddings = model(images)  # Nhận nhúng cho gói hình ảnh
            embeddings.append(batch_embeddings)
            labels.append(batch_labels)

    embeddings = torch.cat(embeddings, dim=0)
    labels = torch.cat(labels, dim=0)
    print('Embedding are ready')
    return embeddings, labels


def average_embeddings_by_label(embeddings, labels):
    unique_labels = labels.unique()  # Nhận thẻ duy nhất
    averaged_embeddings = torch.zeros((len(unique_labels), embeddings.size(1))).to(device)  # Tensor để lưu trữ các nhúng trung bình

    for i, label in enumerate(unique_labels):
        # Nhúng chỉ mục tương ứng với nhãn hiện tại
        indices = (labels == label).nonzero(as_tuple=True)[0]
        # Tính trung bình nhúng cho nhãn hiện tại
        averaged_embeddings[i] = embeddings[indices].mean(dim=0)

    return averaged_embeddings, unique_labels

def get_random_embedding_per_label(model, data_loader):
    model.eval()  # Chuyển mô hình sang chế độ đánh giá
    embeddings_dict = defaultdict(list)

    with torch.no_grad():  # Vô hiệu hóa tính toán gradient
        for images, labels in data_loader:
            images = images.to(device)  # Chuyển hình ảnh sang thiết bị của người mẫu
            embeddings = model(images)  # Nhận nhúng cho gói hình ảnh
            for embedding, label in zip(embeddings, labels):
                embeddings_dict[label.item()].append(embedding)

    # Chọn nhúng ngẫu nhiên cho mỗi thẻ
    random_embeddings = {label: random.choice(embeddings) for label, embeddings in embeddings_dict.items()}

    # Chuyển đổi từ điển thành danh sách nhúng và nhãn tương ứng
    unique_labels = list(random_embeddings.keys())
    random_embedding_list = list(random_embeddings.values())

    # Xếp chồng danh sách nhúng vào một tensor duy nhất
    random_embeddings_tensor = torch.stack(random_embedding_list)

    return random_embeddings_tensor, unique_labels

In [ ]:
model = InceptionResnetV1(pretrained='vggface2').eval()
model.to(device)
# random_embeddings, labels = get_random_embedding_per_label(model, train_loader)
embeddings, labels = get_embeddings(model, train_loader)
averaged_embeddings, unique_labels = average_embeddings_by_label(embeddings, labels)

In [ ]:
len(test_loader)
# len(control_loader)

In [ ]:
def evaluate_accuracy(data_loader, model, reference_embeddings, reference_labels, threshold=0.38):
    model.eval()
    correct_matches = 0
    total_samples = 0
    all_labels = []
    distances = []
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            batch_embeddings = model(images)

            for i, input_embedding in enumerate(batch_embeddings):
                input_embedding = input_embedding.squeeze()
                min_distance = float('inf')
                min_index = None

                # Tìm kiếm thông qua tất cả các nhúng tham chiếu và tìm nhúng gần nhất
                for j, ref_embedding in enumerate(reference_embeddings):
                    distance = cosine(input_embedding.cpu().numpy(), ref_embedding.cpu().numpy())
                    if distance < min_distance:
                        min_distance = distance
                        min_index = j

                distances.append(min_distance)

                if len(data_loader) == 10:
                # Kiểm tra xem khoảng cách tìm thấy có vượt quá ngưỡng không
                    if min_distance < threshold:
                        predicted_label = reference_labels[min_index]
                    else:
                        predicted_label = None

                    # Đếm các kết quả phù hợp chính xác
                    if predicted_label == labels[i]:
                        correct_matches += 1
                    total_samples += 1
#             all_labels.extend(labels.tolist())
#     # Tính toán độ chính xác
#     accuracy = correct_matches / total_samples
                else:
                    if min_distance < threshold:
                        correct_matches += 1
                        # Подсчет правильных совпадений

                    total_samples += 1
#             all_labels.extend(labels.tolist())
#         # Tính toán độ chính xác
#     accuracy = correct_matches / total_samples
            all_labels.extend(labels.tolist())
    # Tính toán độ chính xác
    accuracy = correct_matches / total_samples
    return accuracy, all_labels, distances

Những người có trong cơ sở dữ liệu

In [ ]:
accuracy, labels_faces, distances = evaluate_accuracy(test_loader, model, averaged_embeddings, unique_labels)
print(f'Độ chính xác nhận dạng: {accuracy}')

Những người không có trong cơ sở dữ liệu

In [ ]:
accuracy, labels_faces, distances = evaluate_accuracy(control_loader, model, averaged_embeddings, unique_labels)
print(f'Tỷ lệ lỗi dự đoán: {accuracy}')

In [ ]:
import matplotlib.pyplot as plt
# Vẽ đồ thị
plt.scatter(distances, labels_faces)
plt.xlabel('Distance')
plt.ylabel('Data')
plt.title('Cosine Distances and Data')
plt.show()

#  Trong thực tế, việc bảo mật dữ liệu trong hộ chiếu điện tử cần sử dụng các kỹ thuật mã hóa và xác thực phức tạp hơn nhiều.
Có thể kể đến như


1. Mã hóa dữ liệu


Mã hóa đối xứng


Một ví dụ về mã hóa và giải mã dữ liệu một cách an toàn AES ở GCM :


In [ ]:
import os

from cryptography.hazmat.primitives.ciphers import (
    Cipher, algorithms, modes
)

def encrypt(key, plaintext, associated_data):
    # Generate a random 96-bit IV.
    iv = os.urandom(12)

    # Construct an AES-GCM Cipher object with the given key and a
    # randomly generated IV.
    encryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv),
    ).encryptor()

    # associated_data will be authenticated but not encrypted,
    # it must also be passed in on decryption.
    encryptor.authenticate_additional_data(associated_data)

    # Encrypt the plaintext and get the associated ciphertext.
    # GCM does not require padding.
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()

    return (iv, ciphertext, encryptor.tag)

def decrypt(key, associated_data, iv, ciphertext, tag):
    # Construct a Cipher object, with the key, iv, and additionally the
    # GCM tag used for authenticating the message.
    decryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv, tag),
    ).decryptor()

    # We put associated_data back in or the tag will fail to verify
    # when we finalize the decryptor.
    decryptor.authenticate_additional_data(associated_data)

    # Decryption gets us the authenticated plaintext.
    # If the tag does not match an InvalidTag exception will be raised.
    return decryptor.update(ciphertext) + decryptor.finalize()

iv, ciphertext, tag = encrypt(
    key,
    b"a secret message!",
    b"authenticated but not encrypted payload"
)

print(decrypt(
    key,
    b"authenticated but not encrypted payload",
    iv,
    ciphertext,
    tag
))

Mã hóa xác thực

Cấu trúc ChaCha20Poly1305 được xác định trong RFC 7539. Đó là mật mã luồng kết hợp với MAC mang lại sự đảm bảo tính toàn vẹn mạnh mẽ.


In [ ]:
import os
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
data = b"a secret message"
aad = b"authenticated but unencrypted data"
key = ChaCha20Poly1305.generate_key()
chacha = ChaCha20Poly1305(key)
nonce = os.urandom(12)
ct = chacha.encrypt(nonce, data, aad)
chacha.decrypt(nonce, ct, aad)

2. Xác thực(xác thực bằng tin nhắn)

Mã xác thực tin nhắn dựa trên mật mã (CMAC)

In [ ]:
from cryptography.hazmat.primitives import cmac
from cryptography.hazmat.primitives.ciphers import algorithms
c = cmac.CMAC(algorithms.AES(key))
c.update(b"message to authenticate")
c.finalize()

Kiểm tra xem chữ ký đã cho có đúng hay không

In [ ]:
from cryptography.hazmat.primitives import cmac
from cryptography.hazmat.primitives.ciphers import algorithms
c = cmac.CMAC(algorithms.AES(key))
c.update(b"message to authenticate")
c.verify(b"an incorrect signature")
## b'\xe6\xbaG\xdc\xd5\x7f\xb1A\xd1\xfb\x8e\xf9Ml^?' thay dòng này vào verify vì nó là mã tin nhắn đúng

Mã xác thực tin nhắn dựa trên hàm băm (HMAC)

In [ ]:
from cryptography.hazmat.primitives import hashes, hmac
key = b'test key. Beware! A real key should use os.urandom or TRNG to generate'
h = hmac.HMAC(key, hashes.SHA256())
h.update(b"message to hash")
signature = h.finalize()
signature

Kiểm tra xem chữ ký đã cho có đúng hay không



In [ ]:
h = hmac.HMAC(key, hashes.SHA256())
h.update(b"message to hash")
h_copy = h.copy() # get a copy of `h' to be reused
h.verify(signature)

h_copy.verify(b"an incorrect signature")
## k\xd9\xb29\xefS\xf8\xcf\xec\xed\xbf\x95\xe6\x97X\x18\x9e%\x11DU1\x9fq}\x9a\x9c\xe0)y`=